# **INSTALL AND IMPORT LIBRARIES**

In [ ]:
!pip install -q tensorflow scikit-learn matplotlib
!pip install cryptography
!pip install -q flwr
!pip install -q ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.5/542.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.2 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.2 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# **PREPROCESSING THE DATA**

In [4]:
# Load dataset
df = pd.read_csv("hmnist_64_64_L.csv")
print("Dataset shape:", df.shape)
print("Missing labels:", df['label'].isnull().sum())
print("Unique labels:", df['label'].unique())

# Check the structure
print(df.shape)
df.head()

Dataset shape: (5000, 4097)
Missing labels: 0
Unique labels: [2 5 7 6 8 1 4 3]
(5000, 4097)


,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel4087,pixel4088,pixel4089,pixel4090,pixel4091,pixel4092,pixel4093,pixel4094,pixel4095,label
0,134,99,119,130,142,169,152,139,117,87,...,112,89,73,100,120,120,126,140,195,2
1,55,64,74,63,74,75,71,73,70,77,...,79,85,86,77,68,66,65,68,69,2
2,114,116,136,152,132,100,151,150,127,205,...,128,157,159,205,182,143,129,89,122,2
3,86,82,88,85,103,93,98,109,104,115,...,79,80,109,128,89,85,80,63,48,2
4,168,143,140,139,129,123,123,141,137,101,...,231,199,183,195,179,134,142,158,149,2


In [5]:
# Label and image preprocessing
labels = df['label'].astype(int).values - 1  # Convert 1–8 to 0–7
images = df.drop('label', axis=1).values
# Convert to grayscale 64x64x1 and normalize
images = images.reshape(-1, 64, 64, 1).astype('float16') / 255.0  # use float16 to reduce memory


In [6]:
# Split data equally among 3 clients (each gets all 8 classes)
client_data = {i: {"x": [], "y": []} for i in range(3)}
for class_label in np.unique(labels):
    indices = np.where(labels == class_label)[0]
    np.random.shuffle(indices)
    thirds = np.array_split(indices, 3)
    for i in range(3):
        client_data[i]["x"].extend(images[thirds[i]])
        client_data[i]["y"].extend(labels[thirds[i]])


In [7]:
# Convert to NumPy arrays
for i in range(3):
    client_data[i]["x"] = np.array(client_data[i]["x"])
    client_data[i]["y"] = np.array(client_data[i]["y"])
    print(f"Client {i} samples: {len(client_data[i]['x'])}")


Client 0 samples: 1672
Client 1 samples: 1664
Client 2 samples: 1664


In [8]:
# Check class balance
for i in range(3):
    print(f"\nClient {i} class distribution:")
    unique, counts = np.unique(client_data[i]["y"], return_counts=True)
    for label, count in zip(unique, counts):
        print(f"Class {label}: {count}")



Client 0 class distribution:
Class 0: 209
Class 1: 209
Class 2: 209
Class 3: 209
Class 4: 209
Class 5: 209
Class 6: 209
Class 7: 209

Client 1 class distribution:
Class 0: 208
Class 1: 208
Class 2: 208
Class 3: 208
Class 4: 208
Class 5: 208
Class 6: 208
Class 7: 208

Client 2 class distribution:
Class 0: 208
Class 1: 208
Class 2: 208
Class 3: 208
Class 4: 208
Class 5: 208
Class 6: 208
Class 7: 208


In [9]:
# Focal Loss (handles class imbalance better than sparse_categorical_crossentropy)
import tensorflow.keras.backend as K
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=8)
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return K.sum(loss, axis=1)
    return focal_loss_fixed

# **BUILDING THE MODEL - CNN**

In [10]:
# Define CNN model
def create_cnn_model():
    model = Sequential([
        Input(shape=(64, 64, 1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(8, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss=focal_loss(gamma=2., alpha=0.25),  # using focal loss right here
        metrics=['accuracy']
    )
    return model


# **TESTING THE ACCURACY BEFORE FEDERATED LEARNING**

In [11]:
# Test locally on 1 client
model = create_cnn_model()
history = model.fit(
    client_data[0]["x"], client_data[0]["y"],
    epochs=20,
    batch_size=16,
    validation_split=0.1
)


Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 123ms/step - accuracy: 0.2770 - loss: 0.3315 - val_accuracy: 0.0000e+00 - val_loss: 0.4304
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 115ms/step - accuracy: 0.4465 - loss: 0.1980 - val_accuracy: 0.0000e+00 - val_loss: 0.3930
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 21s 118ms/step - accuracy: 0.5126 - loss: 0.1621 - val_accuracy: 0.0000e+00 - val_loss: 0.1352
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 118ms/step - accuracy: 0.5636 - loss: 0.1521 - val_accuracy: 0.0000e+00 - val_loss: 0.1795
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 0.5781 - loss: 0.1429 - val_accuracy: 0.0000e+00 - val_loss: 0.1519
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.5915 - loss: 0.1339 - val_accuracy: 0.0000e+00 - val_loss: 0.2053
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 114ms/step - accuracy: 0.6273 - loss: 0.1198 - val_accuracy: 0.0000e+00 - val_loss: 0.1774
Epoch 8/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 115ms/step - accuracy: 0.65

# **FEDERATED LEARNING PLAN**

In [12]:
# Prepare Federated Client
import flwr as fl
from sklearn.model_selection import train_test_split

class HistologyClient(fl.client.NumPyClient):
    def __init__(self, x_train, y_train, x_test, y_test):
        self.model = create_cnn_model()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        early_stopping = tf.keras.callbacks.EarlyStopping(
            patience=2, restore_best_weights=True, monitor="val_loss"
        )
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, min_lr=1e-6)
        self.model.fit(self.x_train, self.y_train,
                       validation_data=(self.x_test, self.y_test),
                       epochs=20, batch_size=16,
                       verbose=0, callbacks=[early_stopping, reduce_lr])
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}

# Split each client's data into train/test
client_train_test = {}
for i in range(3):
    x = client_data[i]["x"]
    y = client_data[i]["y"]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)
    client_train_test[i] = (x_train, y_train, x_test, y_test)

In [13]:
# Global evaluation function
def get_evaluate_fn():
    all_x = np.concatenate([client_train_test[i][2] for i in range(3)])
    all_y = np.concatenate([client_train_test[i][3] for i in range(3)])
    def evaluate(server_round, parameters, config):
        model = create_cnn_model()
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(all_x, all_y, verbose=0)
        print(f"[Round {server_round}] 🔍 Global Accuracy: {accuracy:.4f}")
        return loss, {"accuracy": accuracy}
    return evaluate

# Client creation factory
def client_fn(cid: str):
    cid = int(cid)
    x_train, y_train, x_test, y_test = client_train_test[cid]
    return HistologyClient(x_train, y_train, x_test, y_test)

# Start Federated Learning
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=fl.server.strategy.FedAvg(evaluate_fn=get_evaluate_fn()),
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2025-04-22 19:21:19,230	INFO worker.py:1852 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resource

[Round 0] 🔍 Global Accuracy: 0.1279


(ClientAppActor pid=10021) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=10021) 
(ClientAppActor pid=10021)             This is a deprecated feature. It will be removed
(ClientAppActor pid=10021)             entirely in future versions of Flower.
(ClientAppActor pid=10021)         
(ClientAppActor pid=10021) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=10020) 
(ClientAppActor pid=10020)         
(ClientAppActor pid=10021) 
(ClientAppActor pid=10021)         
(ClientAppActor pid=10021) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context

[Round 1] 🔍 Global Accuracy: 0.4076


(ClientAppActor pid=10021) 
(ClientAppActor pid=10021)         
(ClientAppActor pid=10021) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=10021)             This is a deprecated feature. It will be removed
(ClientAppActor pid=10021)             entirely in future versions of Flower.
(ClientAppActor pid=10020) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=10020) 
(ClientAppActor pid=10020)             This is a deprecated feature. It will be removed
(ClientAppActor pid=10020)             entirely in future versions of Flower.
(ClientAppActor pid=1002

...

[Round 9] 🔍 Global Accuracy: 0.7882


(ClientAppActor pid=10021) 
(ClientAppActor pid=10021)         
(ClientAppActor pid=10021) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=10021)             This is a deprecated feature. It will be removed
(ClientAppActor pid=10021)             entirely in future versions of Flower.
(ClientAppActor pid=10021) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=10020) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.c

[Round 10] 🔍 Global Accuracy: 0.7752


(ClientAppActor pid=10020) 
(ClientAppActor pid=10020)         
(ClientAppActor pid=10020) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=10020)             This is a deprecated feature. It will be removed
(ClientAppActor pid=10020)             entirely in future versions of Flower.
(ClientAppActor pid=10020) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=10021) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.c

History (loss, distributed):
	round 1: 0.2520991493682642
	round 2: 0.10854618166471933
	round 3: 0.09863234734856761
	round 4: 0.08643039138047964
	round 5: 0.08059153521989847
	round 6: 0.08007223981541473
	round 7: 0.07953365499174202
	round 8: 0.07265724350403358
	round 9: 0.07329141534678824
	round 10: 0.07440862123604183
History (loss, centralized):
	round 0: 0.39946216344833374
	round 1: 0.2520991563796997
	round 2: 0.10854616016149521
	round 3: 0.09863235801458359
	round 4: 0.08643040060997009
	round 5: 0.08059152960777283
	round 6: 0.08007223159074783
	round 7: 0.079533651471138
	round 8: 0.07265723496675491
	round 9: 0.07329141348600388
	round 10: 0.07440861314535141
History (metrics, centralized):
{'accuracy': [(0, 0.12787212431430817),
              (1, 0.40759241580963135),
              (2, 0.6373626589775085),
              (3, 0.6593406796455383),
              (4, 0.7202796936035156),
              (5, 0.7462537288665771),
              (6, 0.757242739200592),
        